In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
# Cargar datos
raw_data_17 = pd.read_csv('Data/nac2017.csv', encoding='latin')
raw_data_18 = pd.read_csv('Data/nac2018.csv', encoding='latin')
raw_data_21 = pd.read_csv('Data/nac2021.csv', encoding='latin')
raw_data_22 = pd.read_csv('Data/nac2022.csv', encoding='latin')

C:\Users\pablo\AppData\Local\Temp\ipykernel_65480\3193693375.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data_22 = pd.read_csv('Data/nac2022.csv', encoding='latin')


In [9]:
raw_data_21 = raw_data_21.drop(columns='TIPOFORMULARIO')
raw_data_22 = raw_data_22.drop(columns='TIPOFORMULARIO')

In [10]:
raw_data_tot = pd.concat([raw_data_17, raw_data_18, raw_data_21, raw_data_22], ignore_index=True)

In [11]:
raw_data_tot = raw_data_tot.drop(columns=['ATEN_PAR','T_GES_AGRU_CIE','IDPERTET','IDPERTET','CODPRES','N_HIJOSV','IDCLASADMI','ULTCURMAD','ULTCURPAD','PROFESION','EDAD_PADRE','NIV_EDUP'])

In [ ]:
# Información de los datos
raw_data_tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2496358 entries, 0 to 2496357
Data columns (total 27 columns):
 #   Column      Dtype  
---  ------      -----  
 0   COD_DPTO    int64  
 1   COD_MUNIC   int64  
 2   AREANAC     int64  
 3   SIT_PARTO   int64  
 4   OTRO_SIT    object 
 5   SEXO        int64  
 6   PESO_NAC    int64  
 7   TALLA_NAC   int64  
 8   ANO         int64  
 9   MES         int64  
 10  T_GES       int64  
 11  NUMCONSUL   int64  
 12  TIPO_PARTO  int64  
 13  MUL_PARTO   int64  
 14  APGAR1      int64  
 15  APGAR2      int64  
 16  IDHEMOCLAS  int64  
 17  IDFACTORRH  int64  
 18  EDAD_MADRE  int64  
 19  EST_CIVM    int64  
 20  NIV_EDUM    int64  
 21  CODPTORE    float64
 22  CODMUNRE    float64
 23  AREA_RES    float64
 24  FECHA_NACM  object 
 25  N_EMB       int64  
 26  SEG_SOCIAL  int64  
dtypes: float64(3), int64(22), object(2)
memory usage: 514.2+ MB


In [13]:
raw_data_tot.to_csv('Data/nacimientos.csv', index=False, encoding='latin')

Descripción variables numéricas:

In [ ]:
# Descripción variables numéricas
pd.set_option('display.max_columns', None) #Si hay muchas columnas
raw_data_tot.describe()

In [ ]:
# Gráficos para variables numéricas
features_num = ['NUMCONSUL','N_EMB']

plt.figure(figsize=(10, 3))
for i, feature in enumerate(features_num):
    plt.subplot(1, 2, i+1)  # Crear subplots en una cuadrícula de 2 filas y 4 columnas
    sns.histplot(data=raw_data_tot, x=feature, kde=False, bins=10)
    plt.title(f'Histograma de {feature}')
    plt.tight_layout()

plt.show()

Descripción de variables categóricas:

In [ ]:
# Descripción de Variables categóricas
features_cat = raw_data_tot[['SIT_PARTO','OTRO_SIT','SEXO','PESO_NAC','TALLA_NAC','ANO','MES','T_GES','TIPO_PARTO','MUL_PARTO',
                'APGAR1','APGAR2','IDHEMOCLAS','IDFACTORRH','EDAD_MADRE','EST_CIVM','NIV_EDUM','SEG_SOCIAL']]

otras = ['COD_DPTO','COD_MUNIC','AREANAC','CODPTORE','CODMUNRE','AREA_RES','FECHA_NACM']
for i, feature in enumerate(features_cat):
    print(pd.Series(raw_data_tot[feature]).value_counts())

Distribución de la variable objetivo:

In [ ]:
# Categórica
sns.countplot(x='APGAR1', data=raw_data_tot)
plt.title('Distribución de la variable objetivo (APGAR1)')
plt.show()

# Categórica
sns.countplot(x='APGAR2', data=raw_data_tot)
plt.title('Distribución de la variable objetivo (APGAR2)')
plt.show()

### 2.2. Calidad de datos

**Consistencia:** hablar de si los tipos de datos encontrados son consistentes con los del diccionario de datos. Los datos deben ser coherentes con las descripciones recibidas.

**Unicidad:** Hablar de registros duplicados. Cada registro debe ser único, y la presencia de datos duplicados indica un problema de calidad. 

**Completitud:** Hablar de valores faltantes. Es importante que toda la información necesaria esté disponible y no haya campos obligatorios vacíos o incompletos.

**Validez:** Hablar de valores atípicos. Los datos deben cumplir con las reglas del negocio o estar dentro de rangos válidos.
Ejemplo: Si en una columna de edad se encuentran valores negativos o extremadamente altos (como 200 años), estos serían considerados valores atípicos o no válidos.

#### Registros duplicados:

In [ ]:
duplicates = raw_data_tot.duplicated(keep = False).sum()
print(f'Registros duplicados en el conjunto de datos: {duplicates}')

#### Valores faltantes:

In [ ]:
missing_values = raw_data_tot.isnull().sum()
print(f'Valores faltantes en el conjunto de datos: missing_values')

# Porcentajes 
missing_values_p = (missing_values / len(raw_data_tot)) * 100
print(f'Porcentaje de valores faltantes en el conjunto de datos: missing_values_p')

# Visualización de los valores faltantes
sns.heatmap(raw_data_tot.isnull(), cbar=False, cmap='viridis')
plt.title('Mapa de calor de los valores faltantes')
plt.show()

#### Outliers:

In [ ]:
# Porcentaje de outliers
features_outliers = []
print('Número y porcentaje de outliers:')

for column in features_num:
    Q1 = raw_data_tot[column].quantile(0.25)
    Q3 = raw_data_tot[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = raw_data_tot[(raw_data_tot[column] < lower_bound) | (raw_data_tot[column] > upper_bound)]
    percentage = (len(outliers) / len(raw_data_tot[column])) * 100
    
    if percentage > 0:
        features_outliers.append(column)
        
    print(f'{column}: {len(outliers)} | {percentage:.2f}%')

## Limpieza de datos

In [18]:
clean_data = raw_data_tot.copy()

In [19]:
clean_data=clean_data.drop_duplicates().drop(columns=['OTRO_SIT','AREANAC','MES','IDHEMOCLAS', 'IDFACTORRH',
                                'EST_CIVM', 'NIV_EDUM','CODPTORE', 'CODMUNRE', 'AREA_RES'])
clean_data.columns

Index(['COD_DPTO', 'COD_MUNIC', 'SIT_PARTO', 'SEXO', 'PESO_NAC', 'TALLA_NAC',
       'ANO', 'T_GES', 'NUMCONSUL', 'TIPO_PARTO', 'MUL_PARTO', 'APGAR1',
       'APGAR2', 'EDAD_MADRE', 'FECHA_NACM', 'N_EMB', 'SEG_SOCIAL'],
      dtype='object')

In [23]:
cols_99=['APGAR1', 'APGAR2', 'EDAD_MADRE', 'N_EMB', 'NUMCONSUL']
cols_9 = ['SIT_PARTO', 'PESO_NAC', 'TALLA_NAC', 'T_GES', 'TIPO_PARTO', 'MUL_PARTO', 'SEG_SOCIAL']
cols_3= ['SEXO']
cols_4=['TIPO_PARTO']
cols_6 = ['T_GES']

for col99 in cols_99:
    clean_data = clean_data[clean_data[col99] != 99]

for col9 in cols_9:
    clean_data= clean_data[clean_data[col9] != 9]

for col3 in cols_3:
    clean_data= clean_data[clean_data[col3] != 3]

for col4 in cols_4:
    clean_data= clean_data[clean_data[col4] != 4]

for col6 in cols_6:
    clean_data= clean_data[clean_data[col6] != 6]

In [ ]:
#datos=clean_data.dropna()

In [25]:
clean_data.shape

(2441519, 17)

In [26]:
clean_data.to_csv('Data/nacimientos_clean.csv', index=False, encoding='latin')

### Datos mapa

In [27]:
import pandas as pd

clean_data = pd.read_csv('Data/nacimientos_clean.csv', sep=',', encoding='latin')

In [28]:
# Cargar el archivo CSV cods con los tipos de datos correctos para las columnas
columnas_str = ['COD_Departamento', 'COD_Municipio']
df_codigos = pd.read_csv('Data/cods_dpto_munic.csv', sep=',', encoding='latin', dtype={col: str for col in columnas_str})
df_codigos.shape

(1122, 4)

In [29]:
# Asegurarse de que datos tenga las columnas COD_DPTO y COD_MUNIC con ceros a la izquierda
clean_data['COD_DPTO'] = clean_data['COD_DPTO'].astype(str).str.zfill(2)
clean_data['COD_MUNIC'] = clean_data['COD_MUNIC'].astype(str).str.zfill(3)

# Crear una columna concatenada en datos
clean_data['COD_COMBINADO'] = clean_data['COD_DPTO'] + clean_data['COD_MUNIC']

In [30]:
# Realizar el merge usando las columnas concatenadas
df_resultado = pd.merge(clean_data, df_codigos, left_on='COD_COMBINADO', right_on='COD_Municipio', how='left')

# Seleccionar las columnas de interés
df_resultado = df_resultado.drop(columns=['COD_Departamento', 'COD_Municipio','COD_DPTO','COD_MUNIC'])

# Guardar el resultado si es necesario
df_resultado.to_csv('Data/nacimientos_full.csv', index=False, encoding='latin')

# Ver el resultado
df_resultado.shape

(2441519, 18)